#### A comparative study with X-Wines and Movielens[1] datasets with recommender systems metrics
#### Rogério Xavier de Azambuja (rogerio.xavier@farroupilha.ifrs.edu.br)
#### Dataset X-Wines from https://github.com/rogerioxavier/X-Wines

In [1]:
# Opening required packages for processing
# basic libs
import sys
import pandas as pd
import numpy as np
import random
from tqdm.notebook import tqdm

# Cornac[2] framework
import cornac
from cornac.eval_methods import RatioSplit
from cornac.models import MF, WMF, SVD, UserKNN
from cornac.metrics import Precision, Recall, NDCG, AUC, MAP, MRR

import tensorflow as tf

print(f"System version: {sys.version}")
print(f"Cornac version: {cornac.__version__}")
print(f"Tensorflow version: {tf.__version__}")

System version: 3.7.15 (default, Nov  9 2022, 10:44:37) [MSC v.1916 64 bit (AMD64)]
Cornac version: 1.14.2
Tensorflow version: 1.15.0


In [2]:
# Opening X-Wines dataset
wines   = pd.read_csv("XWines_100K_wines.csv", low_memory=False, encoding="utf-8", memory_map=True)
ratings = pd.read_csv("XWines_21M_ratings.csv", low_memory=False, encoding="utf-8", memory_map=True)
len(wines), len(ratings)

(100646, 21013536)

In [3]:
print("Total wines:", wines.WineID.nunique(), "from", wines.Code.nunique() ,"different countries")
print("Total users:", ratings.UserID.nunique(), "who rated the wines")
print(f"             in the period between {ratings.Date.min()} and {ratings.Date.max()}")
print("Total ratings:", len(ratings), f"5-stars on wine vintages since {ratings.Vintage.min()}, including non-vintage 'N.V.'")

Total wines: 100646 from 62 different countries
Total users: 1056079 who rated the wines
             in the period between 2012-01-03 08:20:53 and 2021-12-31 23:59:56
Total ratings: 21013536 5-stars on wine vintages since 1950, including non-vintage 'N.V.'


In [4]:
# A first segmentation because the dataset is too big
code = 'FR'
country = 'France'
wines = wines.sort_values(by='Ratings', ascending=False) #####
winesCode = wines.loc[ wines.Code==code ]
print(winesCode.WineID.nunique(), f"wines from the country {country}({code}) ")

24399 wines from the country France(FR) 


In [ ]:
# Open the sample or select new random sample
# sample = pd.read_csv('../RecSys/SampleFR01.csv', low_memory=False, encoding="utf-8", memory_map=True)

In [5]:
# Select rating relationship to sample
n_wines=1682
n_users=943

slide = random.randint(0, 100)  # 5, 10, 50, ...

winesSample = winesCode.WineID.unique()[slide:n_wines+slide]
print(len(winesSample), "selected wines")

ratings = ratings.sort_values(by='Date', ascending=False)

unic_ratings = ratings.drop_duplicates(subset=['UserID','WineID'], keep = 'first')
print(len(unic_ratings), 'unique ratings')

unic_ratings = unic_ratings[unic_ratings.WineID.apply(lambda a: True if a in winesSample else False)]
print(len(unic_ratings), 'segmented wine ratings')

u = unic_ratings.groupby('UserID', as_index=False)[['Rating']].count()
u = u.sort_values(by='Rating', ascending=False)
usersSample = u[:n_users].UserID.tolist()
print(len(usersSample), "selected users")

unic_ratings = unic_ratings[unic_ratings.UserID.apply(lambda a: True if a in usersSample else False)]
print(len(unic_ratings), 'segmented winesXusers ratings')
sample=[]
for _, row in tqdm(unic_ratings.iterrows(), total=unic_ratings.shape[0]):
    # One more verification winesXusers relationship
    if row.WineID in winesSample and row.UserID in usersSample:
        sample.append([row.UserID, row.WineID, row.Rating])
        
sample=pd.DataFrame(sample, columns=['UserID', 'WineID', 'Rating'])
sample= sample.drop_duplicates(subset=['UserID', 'WineID'])
print("Sample ratings:", len(sample), " # users:", len(sample.UserID.unique()), " # wines:",len(sample.WineID.unique()))

1682 selected wines
20590800 unique ratings
1930480 segmented wine ratings
943 selected users
100061 segmented winesXusers ratings


  0%|          | 0/100061 [00:00<?, ?it/s]

Sample ratings: 100061  # users: 943  # wines: 1654


In [6]:
# Adjust if necessary: delete one rating for each user who rated the most
n_cut = 61
for user in usersSample[:n_cut]:
    i = sample[sample.UserID==user][:1].index
    sample = sample[sample.index!=i.values[0]]

In [7]:
print("Sample ratings:", len(sample), " # users:", len(sample.UserID.unique()), " # wines:",len(sample.WineID.unique()))
SEED = 765
VERBOSE = False
RATING_THRESHOLD = 4.0
TEST_SIZE = 0.2
TRAIN_SIZE = 1 - TEST_SIZE

Sample ratings: 100000  # users: 943  # wines: 1673


In [8]:
# load the built-in MovieLens 100K and split the data based on ratio
ml_100k = cornac.datasets.movielens.load_feedback()
rsML = RatioSplit(data=ml_100k, test_size=TEST_SIZE, rating_threshold=RATING_THRESHOLD, exclude_unknowns=True, seed=SEED)

# Movielens list format: userId, movieId and rating 5-stars for 100,000 ratings
ml_100k[:5], len(ml_100k)

([('196', '242', 3.0),
  ('186', '302', 3.0),
  ('22', '377', 1.0),
  ('244', '51', 2.0),
  ('166', '346', 1.0)],
 100000)

In [9]:
# X-Wines in same list format
XW_100k = []
for _,row in sample.iterrows():
    XW_100k.append(eval(f"('{int(row.UserID)}', '{int(row.WineID)}', {float(row.Rating)})"))
# split the X-Wines dataset based on ratio   
rsXW = RatioSplit(data=XW_100k, test_size=TEST_SIZE, rating_threshold=RATING_THRESHOLD, exclude_unknowns=True, seed=SEED)

XW_100k[:5], len(XW_100k)

([('1155087', '111497', 4.0),
  ('1120177', '111418', 4.5),
  ('1006433', '111754', 4.5),
  ('1174755', '111584', 4.5),
  ('1140347', '111584', 4.0)],
 100000)

In [12]:
# define models
K=10
LEARNING_RATE = 0.001

models = [
    MF (learning_rate=LEARNING_RATE, seed=SEED, verbose=VERBOSE),
    WMF(learning_rate=LEARNING_RATE, seed=SEED, verbose=VERBOSE),
    SVD(k=2, learning_rate=0.0014, lambda_reg=0.08, seed=SEED, verbose=VERBOSE), #tuned parameters
    UserKNN(k=K, similarity="cosine",  name="UserKNN-Cosine", verbose=VERBOSE),
    UserKNN(k=K, similarity="pearson", name="UserKNN-Pearson", mean_centered=True, verbose=VERBOSE),
]

# define metrics to evaluate the models
metrics=[ Precision(k=[5, 10, 100]), Recall(k=[5, 10, 100]), NDCG(k=[5, 10, 100]), AUC(), MAP(), MRR()]

In [11]:
# put it together in an experiment, voilà!
cornac.Experiment(eval_method=rsML, models=models, metrics=metrics, user_based=True, ).run()


TEST:
...
                |    AUC |    MAP |    MRR | NDCG@10 | NDCG@100 | NDCG@5 | Precision@10 | Precision@100 | Precision@5 | Recall@10 | Recall@100 | Recall@5 | Train (s) | Test (s)
--------------- + ------ + ------ + ------ + ------- + -------- + ------ + ------------ + ------------- + ----------- + --------- + ---------- + -------- + --------- + --------
MF              | 0.7254 | 0.0479 | 0.1540 |  0.0598 |   0.1435 | 0.0558 |       0.0526 |        0.0349 |      0.0534 |    0.0453 |     0.2894 |   0.0191 |    0.0544 |   1.1619
WMF             | 0.9350 | 0.0675 | 0.1198 |  0.0421 |   0.2247 | 0.0315 |       0.0374 |        0.0507 |      0.0298 |    0.0424 |     0.5729 |   0.0117 |    5.5188 |   0.9137
SVD             | 0.7232 | 0.0458 | 0.1406 |  0.0562 |   0.1322 | 0.0475 |       0.0502 |        0.0322 |      0.0441 |    0.0442 |     0.2640 |   0.0161 |    0.0313 |   1.2101
UserKNN-Cosine  | 0.6960 | 0.0162 | 0.0150 |  0.0005 |   0.0374 | 0.0000 |       0.0005 |        0.0118 

In [13]:
# put it together in an experiment, voilà! Maybe you need run define models before once again
cornac.Experiment(eval_method=rsXW, models=models, metrics=metrics, user_based=True).run()


TEST:
...
                |    AUC |    MAP |    MRR | NDCG@10 | NDCG@100 | NDCG@5 | Precision@10 | Precision@100 | Precision@5 | Recall@10 | Recall@100 | Recall@5 | Train (s) | Test (s)
--------------- + ------ + ------ + ------ + ------- + -------- + ------ + ------------ + ------------- + ----------- + --------- + ---------- + -------- + --------- + --------
MF              | 0.7478 | 0.0488 | 0.1628 |  0.0583 |   0.1406 | 0.0598 |       0.0534 |        0.0355 |      0.0585 |    0.0397 |     0.2520 |   0.0234 |    0.0520 |   1.2438
WMF             | 0.8333 | 0.0467 | 0.1261 |  0.0385 |   0.1445 | 0.0376 |       0.0378 |        0.0406 |      0.0367 |    0.0235 |     0.2868 |   0.0112 |    5.7322 |   0.9000
SVD             | 0.7545 | 0.0445 | 0.1540 |  0.0512 |   0.1336 | 0.0508 |       0.0483 |        0.0349 |      0.0478 |    0.0343 |     0.2465 |   0.0172 |    0.0157 |   1.3398
UserKNN-Cosine  | 0.7714 | 0.0341 | 0.0685 |  0.0200 |   0.1047 | 0.0150 |       0.0225 |        0.0305 

References:
[1] Movielens by Grouplens/University of Minnesota at https://grouplens.org/datasets/movielens
[2] Cornac is a comparative framework for multimodal recommender systems at https://cornac.preferred.ai